This Notebook is for plotting parts of the PAP algorithm, to compare with the original paper by Pouchou and Pichoir (1991).

In [1]:
import hyperspy.api as hs
import numpy as np
import plotly.graph_objects as go

from PAP_functions.PAP_helper_functions import theoretical_energy, get_C_A_Z_arrays
from PAP_functions.PAP_area_F import set_m_small, ionization_cross_section_Q, mean_atomic_mass_M, mean_ionzation_potential_J, energy_dependent_terms_f_of_v, energy_loss_dE_drhos, whole_dE_drhos, deceleration_factor_one_over_S

In [10]:
hs.material.elements.Ga.Atomic_properties.Xray_lines.Ka['energy (keV)']

9.2517

In [ ]:
## Looking at the values

elements = ['Ga', 'Sb']
line = 'Ga_La'
e0 = 15  # keV
concentrations = [0.5, 0.5]  # at%

e_c = theoretical_energy(line=line)
u = e_c / e0
m_small = set_m_small(line=line)

array_C, array_A, array_Z = get_C_A_Z_arrays(elements=elements, 
                                         concentrations=concentrations, concentration_type='at')
q = ionization_cross_section_Q(e0=e0, line=line)
m = mean_atomic_mass_M(array_C=array_C, array_Z=array_Z, array_A=array_A)
j = mean_ionzation_potential_J(array_C=array_C, array_Z=array_Z, array_A=array_A)
f_of_v = energy_dependent_terms_f_of_v(e=e0, j=j)
dE_drhos = energy_loss_dE_drhos(m=m, j=j, f_of_v=f_of_v)
dE_drhos2 = whole_dE_drhos(array_C=array_C, array_Z=array_Z, array_A=array_A, e0=e0)

print(f'C wt%: {array_C}')
print(f'A Da: {array_A}')
print(f'Z: {array_Z}')
print(f'Q: {q:.3f}')
print(f'M: {m:.3f}')
print(f'J: {j:.3f}')
print(f'1/f(V): {1/f_of_v:.3f}')

In [2]:
def plot_dE_drhos(*, fig, elements, at_list, energies, name):
    array_C, array_A, array_Z = get_C_A_Z_arrays(elements=elements, 
                                         concentrations=at_list, concentration_type='at')
    m = mean_atomic_mass_M(array_C=array_C, array_Z=array_Z, array_A=array_A)
    j = mean_ionzation_potential_J(array_C=array_C, array_Z=array_Z, array_A=array_A)
    f_of_v_list = [energy_dependent_terms_f_of_v(e=e, j=j) for e in energies]
    de_drhos_list = np.array([energy_loss_dE_drhos(m=m, j=j, f_of_v=f) for f in f_of_v_list])
    fig.add_trace(go.Scatter(x=energies, y=de_drhos_list, name=name))


In [13]:
energies = np.linspace(0.01, 50, 10000)
# f_of_v_list = [energy_dependent_terms_f_of_v(e=e, j=j) for e in energies]
# de_drhos_list = np.array([energy_loss_dE_drhos(m=m, j=j, f_of_v=f) for f in f_of_v_list])

fig = go.Figure()
plot_dE_drhos(fig=fig, elements=['Ga', 'Sb'], at_list=[0.5, 0.5], energies=energies, name='GaSb')
plot_dE_drhos(fig=fig, elements=['Ga', 'As'], at_list=[0.5, 0.5], energies=energies, name='GaAs')
plot_dE_drhos(fig=fig, elements=['Cu'], at_list=[1.], energies=energies, name='Cu (PAP Fig. 4)')

fig.update_layout(font=dict(family="EB Garamond SemiBold", size=16, color="black"),
                  xaxis_title="Energy [keV]", xaxis_type="log",
                  yaxis_title="dE/d\u03C1s [keV cm<sup>2</sup>/g]",
                  margin=dict(l=5, r=5, b=5, t=5), width=800, height=400,
                  legend=dict(x=0.995, y=0.99, xanchor='right',),
                  ) 

In [15]:
# fig.write_image('../mastersthesis/figures/PAP_energy_loss_dE_drhos.pdf')
# # fig.write_image('../mastersthesis/figures/PAP_deceleration_of_electrons.pdf')  # old name
